In [6]:
import altair as alt
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator


In [79]:
all_treasuries = pd.read_csv(
    "https://home.treasury.gov/system/files/276/yield-curve-rates-1990-2021.csv"
)

In [80]:
all_treasuries.Date = pd.to_datetime(all_treasuries.Date)


In [109]:
def interpolate_curve(curve):
    curve = curve.iloc[1:]
    curve.columns = ["tenor", "value"]
    curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]

    curve = curve.dropna()

    g = PchipInterpolator(curve["tenor"], curve["value"], extrapolate=True)
    tenors = np.linspace(0, 360, 61, endpoint=True)
    values = g(tenors)

    return tenors, values


In [110]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')


tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["yield"] = curve_data["yield"] / 100.0
curve_data["maturity"] = curve_data["maturity"] / 12


/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_57920/4098965440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curve.loc[:, "tenor"] = [1, 2, 3, 6, 12, 24, 36, 60, 90, 120, 240, 360]


In [111]:
alt.Chart(curve_data).mark_line().encode(
    x="maturity", y="yield", tooltip=["maturity", "yield"]
)


alt.Chart(...)

In [ ]:
old_treasury = all_treasuries.iloc[100].reset_index()
curve_date = all_treasuries.iloc[100].Date.strftime('%Y-%m-%d')

tenors, values = interpolate_curve(old_treasury)

curve_data = pd.DataFrame(
    {"tenor": np.linspace(0, 360, 61, endpoint=True), "value": values}
)

curve_data["tenor"] = curve_data["tenor"] / 12
curve_data["value"] = curve_data["value"] / 100.0


In [113]:
curve_values = list(
    zip(
        np.tile("us_treasury", curve_data.shape[0]),
        np.tile(curve_date, curve_data.shape[0]),
        curve_data.maturity,
        curve_data["yield"],
    )
)


In [ ]:
for i in curve_values:
    query = """
        INSERT INTO curve
        (curve_name, date, tenor, value)
        VALUES (%s,%s,%s,%s)
        """
    